# Demo of using MAD for Domain Adaptation

This notebook illustrates the use of the MAD loss in a domain adaptation setting, as described in the paper.
Note that this notebook relies on a simplified adaptation problem coined _TinyTimeMatch_ in order to have a simple-to-reproduce example of our method.
TinyTimeMatch is a subsample of the DK1 $\rightarrow$ AT1 adaptation problem from miniTimeMatch.

In [1]:
import torch.nn as nn
import os
import numpy as np
import torch
import warnings

from dataset_loader import load_a_dataset
from General_model import CNNMAD

2023-03-31 14:37:00.795497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


We define the MAD-CNN procedure to train models for domain adaptation for time series classification.

Here, we will try to classify series from Austria in the "MiniTimeMatch" dataset and we will use for training series from Denmark.
For these domains, the corresponding numbers in the dataset are 4 for Austria and 3 for Denmark. 
They both have 7 classes and the series are composed of 10 features

First we have to load the datasets:

In [2]:
source = 3
target = 4

train_source, train_source_label, valid_source, valid_source_label, test_source, test_source_label = load_a_dataset(dataset_name="Dataset/TinyTimeMatch/TinyTimeMatch", domain_id=str(source))
train_target, train_target_label, valid_target, valid_target_label, test_target, test_target_label = load_a_dataset(dataset_name="Dataset/TinyTimeMatch/TinyTimeMatch", domain_id=str(target))

number_of_features = 10
number_of_classes = len(set(train_source_label.tolist()))

Then, we have to define the networks we want to use.
We need a feature extractor that will be composed of 3 layers and a classifier that will be a single layer.

In [3]:
feature_extractor = nn.Sequential(
            nn.Conv1d(in_channels=number_of_features, out_channels=128, kernel_size=8, stride=1, padding="same", bias=False),
            nn.BatchNorm1d(num_features=128, affine=False),
            nn.ReLU(),

            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding="same", bias=False),
            nn.BatchNorm1d(num_features=256, affine=False),
            nn.ReLU(),

            nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding="same", bias=False),
            nn.BatchNorm1d(num_features=128, affine=False),
            nn.ReLU())
classifier = nn.Sequential(nn.Linear(128, number_of_classes))


We need to define hyper-parameters such as $\alpha=0.1$, $\beta=0.01$, the learning rate $=0.001$ and the batchsize $=256$.
We took the values from the papers.

We also need to define some paths and names for saving purposes.

In [4]:
alpha = 0.1
beta = 0.01
learning_rate = 0.001
batchsize = 256
number_of_iterations = 100
name_model = "MAD_demo"

path_save = os.path.join("TinyTimeMatch", "MAD", str(alpha), str(beta), str(learning_rate), str(source) + "_" + str(target))

if os.path.exists(path_save) is False:
    os.makedirs(path_save)
name = path_save + "/" + name_model

In case we want weakly supervised learning, we have to find the proportion of each class in the target domain.

In [5]:
_, c = np.unique(train_target_label, return_counts=True)
target_prop = c / train_target.shape[0]
target_prop = torch.Tensor(target_prop)

The batchsize has to be carefully calibrated in case one of the two domains has less series than the batchsize chosen by the user.
To make sure everything goes smoothly, the actual batchsize is set to be equal to the minimum between the desired batchsize and the number of series in both domains.

In [6]:
source_size = train_source.shape[0]
target_size = train_target.shape[0]
batchsize = torch.min(torch.tensor([batchsize, source_size, target_size]))
batchsize = batchsize.item()

We can define the whole MAD-CNN model now

In [7]:
CNN_mod = CNNMAD(name=name, 
                 batchsize=batchsize, 
                 feature_extractor=feature_extractor, 
                 classifier=classifier,
                 X_target=train_target,

                 y_target=train_target_label, 
                 lr=learning_rate, 
                 saving=True, #To be set to True if we want to save resutls
                 max_iterations=number_of_iterations,
                 validation_step=1000,

                 target_prop=target_prop, 
                 alpha=alpha, 
                 beta=beta, 
                 MAD_class=True, #To be set to True if we want C-MAD or to False if we want MAD
                 save_OT_plan=False,

                 save_DTW_matrices=False,
                 save_latent_source=False,
                 save_latent_target=False)

We train the model over a few iterations

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    CNN_mod.fit(X_source=train_source, 
                y_source=train_source_label,
                X_source_valid=valid_source,
                y_source_valid=valid_source_label,
                X_target_valid=valid_target)

And we then show the accuracy

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    CNN_mod.evaluate(inputs=test_target, labels=test_target_label)

TinyTimeMatch/MAD/0.1/0.01/0.001/3_4/MAD_demo
100 Evaluation set  target :
Average loss: 0.0134, Accuracy: 39/140 (27.857%)
F1 micro score is :  0.2785714285714286
F1 macro score is :  0.222966073697153
F1 weigthed score is :  0.222966073697153


Here, the accuracy is not expected to be in the same magnitude as the numbers presented in the paper since the method is run for a small number of epochs on a demo subset of an adaptation problem from miniTimeMatch.

Running the exact same experiment outside of a notebook can be done using the following command:

```python3 main.py --dataset "TinyTimeMatch" --model "MAD" --source_id 3 --target_id 4 --alpha 0.1 --beta 0.01 --learning_rate 0.0001 --iteration 10 --name_model "MAD_demo" --seed 100 --target_class_proportion "True" --batchsize 256 --per_class_dtw "True"```